# LangSmith API Key 로드

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

# Langsmith로 프로젝트 추적 설정

In [2]:
from utils import logging

logging.langsmith("OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
OutputParser


# llm 객체 정의

In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

(참고) 아래는 객체 내부의 속성들을 출력할 수 있는 코드입니다.

In [9]:
# print(llm.__dict__) # __dict__ 메서드 사용

{'name': None, 'cache': None, 'verbose': False, 'callbacks': None, 'tags': None, 'metadata': None, 'custom_get_token_ids': None, 'callback_manager': None, 'rate_limiter': None, 'disable_streaming': False, 'client': <openai.resources.chat.completions.Completions object at 0x000002CA48241A10>, 'async_client': <openai.resources.chat.completions.AsyncCompletions object at 0x000002CA48066210>, 'root_client': <openai.OpenAI object at 0x000002CA47CEADD0>, 'root_async_client': <openai.AsyncOpenAI object at 0x000002CA4824E810>, 'model_name': 'gpt-3.5-turbo', 'temperature': 0.7, 'model_kwargs': {}, 'openai_api_key': SecretStr('**********'), 'openai_api_base': None, 'openai_organization': None, 'openai_proxy': None, 'request_timeout': None, 'max_retries': 2, 'presence_penalty': None, 'frequency_penalty': None, 'seed': None, 'logprobs': None, 'top_logprobs': None, 'logit_bias': None, 'streaming': False, 'n': 1, 'top_p': None, 'max_tokens': None, 'tiktoken_model_name': None, 'default_headers': None

In [11]:
# print(f"Model: {llm.model_name}")
# print(f"Temperature: {llm.temperature}")
# print(f"Top-p: {llm.top_p}")
# print(f"Frequency Penalty: {llm.frequency_penalty}")
# print(f"Presence Penalty: {llm.presence_penalty}")

Model: gpt-3.5-turbo
Temperature: 0.7
Top-p: None
Frequency Penalty: None
Presence Penalty: None


# OutputParser

LangChain의 출력파서는 언어 모델(LLM)의 출력을 더 유용하고 구조화된 형태로 변환하는 중요한 컴포넌트입니다.

LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱하고 처리합니다.

출력 파서의 종류에는 Pydantic, 콤마 구분자, JSON, 데이터프레임, 날짜, 열거형, 출력 수정 등이 있습니다.

# 1. PydanticOutputParser

단순 텍스트 형태의 응답 대신, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공 하는 클래스

- get_format_instructions(): 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction) 을 제공

- parse(): 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field



email = """
From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""


## 1.1 출력 파서를 사용하지 않을 경우

In [15]:
template = """
다음 이메일에서 중요한 내용을 추출해주세요.

--이메일--
{email}
"""

prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini")

chain = prompt | llm

answer = chain.invoke({'email': email})
print(answer.content) # 결과str에서 내가 원하는 값만 따로 빼올 수가 없음

이메일의 중요한 내용은 다음과 같습니다:

1. 발신자: 김철수 (바이크코퍼레이션 상무)
2. 수신자: 이은채 (Teddy International)
3. 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. 요청사항:
   - ZENESIS 모델에 대한 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. 제안된 미팅 일정: 다음 주 화요일 (1월 15일) 오전 10시
6. 미팅 장소: 귀사 사무실

이메일의 목적은 ZENESIS 자전거에 대한 유통 협력 가능성을 논의하기 위한 미팅을 제안하고 필요한 정보를 요청하는 것입니다.


## 1.2 Pydantic 스타일로 정의된 클래스를 사용하여 파싱하는 경우

참고로, Field 안에 description 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명이며 LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됩니다.

### 출력 포맷 정의

In [16]:
class EmailSummary(BaseModel): # BaseModel 상속받음
    # 클래스 변수 정의 
    sender: str = Field(description='발신자')
    sender_email: str = Field(description='발신자 이메일 주소')
    recipient: str = Field(description='수신자')
    recipient_email: str = Field(description='수신자 이메일 주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문을 요약한 텍스트')
    date: str = Field(description='메일 본문에 언급된 미팅 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)

### get_format_instructions() 확인

In [17]:
# 원하는 포맷을 얻고싶을 때 이 셀의 결과 값 처럼 JSOn 스키마로 들어가게 됨
print(parser.get_format_instructions()) 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender": {"description": "발신자", "title": "Sender", "type": "string"}, "sender_email": {"description": "발신자 이메일 주소", "title": "Sender Email", "type": "string"}, "recipient": {"description": "수신자", "title": "Recipient", "type": "string"}, "recipient_email": {"description": "수신자 이메일 주소", "title": "Recipient Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날

- get_format_instructions() 실행해서 나온 결과가 사실 프롬프트에 작성돼야할 내용임
- PydanticOutputParser()을 사용하지 않을 경우 get_format_instructions()의 실행 결과를 그대로 프롬프트에 작성해주면 됨 

### 프롬프트 템플릿 작성

In [18]:
prompt = PromptTemplate.from_template(
    """
당신은 유능한 어시스턴트 입니다. 다음 질문에 대해 답변해주세요.

# 사용자 질문
{user_question}

# 이메일
{email}

# 출력 포맷
{format}
"""
)

llm = ChatOpenAI(model_name="gpt-4o")

# format 변수에 포맷 지시사항 담기 
prompt = prompt.partial(format=parser.get_format_instructions())
print(prompt)

input_variables=['email', 'user_question'] input_types={} partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender": {"description": "발신자", "title": "Sender", "type": "string"}, "sender_email": {"description": "발신자 이메일 주소", "title": "Sender Email", "type": "string"}, "recipient": {"description": "수신자", "title": "Recipient", "type": "string"}, "recipient_email": {"description": "수신자 이메일 주소", "title": "Recipient Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메

### chain생성 및 실행

#### invoke() 로 출력

In [19]:
chain = prompt | llm 

response = chain.invoke(
    {
        'user_question': '이메일에서 중요한 내용을 추출해 주세요.',
        'email': email
    }
)

output = response.content
print(output)

```json
{
  "sender": "김철수",
  "sender_email": "chulsoo.kim@bikecorporation.me",
  "recipient": "이은채",
  "recipient_email": "eunchae@teddyinternational.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수가 테디인터내셔널의 이은채에게 신규 자전거 \"ZENESIS\"에 대한 협력을 제안하며, 기술 사양, 배터리 성능, 디자인에 대한 상세 정보를 요청하고, 1월 15일 화요일 오전 10시에 미팅을 제안함.",
  "date": "1월 15일 화요일 오전 10시"
}
```


In [20]:
# output이 str 타입임을 알 수 있음
print(type(output))

<class 'str'>


#### stream() 로 출력

In [21]:
from utils.messages import stream_response

chain = prompt | llm 

answer = chain.stream(
    {
        'user_question': '이메일에서 중요한 내용을 추출해 주세요.',
        'email': email
    }
)

output = stream_response(answer, return_output=True)

```json
{
    "sender": "김철수",
    "sender_email": "chulsoo.kim@bikecorporation.me",
    "recipient": "이은채",
    "recipient_email": "eunchae@teddyinternational.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "김철수 상무는 이은채 대리에게 ZENESIS 자전거에 대한 상세한 브로슈어를 요청하고, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안하였습니다.",
    "date": "1월 15일 오전 10시"
}
```

### parser를 사용하여 결과를 파싱하고 EmailSummary 객체로 변환

In [13]:
structured = parser.parse(output)

In [14]:
structured

EmailSummary(sender='김철수', sender_email='chulsoo.kim@bikecorporation.me', recipient='이은채', recipient_email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무가 이은채 대리에게 바이크코퍼레이션의 ZENESIS 자전거에 대한 상세 브로슈어 요청과 함께 유통 전략 및 마케팅 계획 수립을 위한 정보를 요청하고, 협력 가능성 논의를 위한 미팅을 1월 15일 화요일 오전 10시에 제안함.', date='1월 15일 오전 10시')

In [15]:
# 파라미터로 원하는 데이터에 접근 가능 
structured.sender

'김철수'

## 1.3 parser가 추가된 chain 생성하여 사용할 경우

In [16]:
chain = prompt | llm | parser # parser.parse(output)과 똑같음 

response = chain.invoke(
    {
        'user_question': '이메일에서 주요 내용을 추출해 주세요.',
        'email': email
    }
)
response # str타입을 객체화 하여 각각의 데이터에 instance로 접근할 수 있게 됨

EmailSummary(sender='김철수', sender_email='chulsoo.kim@bikecorporation.me', recipient='이은채', recipient_email='eunchae@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="김철수 상무가 신규 자전거 'ZENESIS'에 대한 브로슈어를 요청하며, 기술 사양, 배터리 성능, 디자인 정보가 필요하다고 설명합니다. 또한, 협력 논의를 위해 1월 15일 화요일 오전 10시에 미팅을 제안합니다.", date='1월 15일 오전 10시')

In [17]:
response.sender

'김철수'

# 2. with_structured_output()

- ChatOpenAI의 with_structured_output()메소드 
- chain에 parser를 추가하지 않음
- 출력을 Pydantic 객체로 변환할 수 있는 메소드
- 단점: stream()지원이 안 됨

### 사용자 입력 context

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

email = """
From: 박지현 (Jihyun.Park@abc.com)
To: 김민재 (Minjae@def.com)
제목: [미팅 제안] 기술 협력을 통한 상호 발전 방안 논의

안녕하세요, 민재 선임님.
저는 DEF 소프트웨어의 연구개발부 팀장 박지현입니다.

최근 전문 저널에서 귀사의 AI 알고리즘에 대한 기사를 읽고 큰 관심을 갖게 되었습니다. 저희 회사의 데이터 분석 기술과 결합하면 시너지 효과를 낼 수 있을 것이라 생각합니다.

이에 대해 논의하기 위해 10월 25일에 미팅을 제안드립니다. 편하신 시간대를 알려주시면 일정에 맞추도록 하겠습니다.

감사합니다.

박지현 드림
"""

출력 포맷 정의

In [19]:
class EmailSummary(BaseModel): # BaseModel 상속받음
    # 클래스 변수 정의 
    sender: str = Field(description='발신자')
    sender_email: str = Field(description='발신자 이메일 주소')
    recipient: str = Field(description='수신자')
    recipient_email: str = Field(description='수신자 이메일 주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문을 요약한 텍스트')
    date: str = Field(description='메일 본문에 언급된 미팅 날짜와 시간')


parser = PydanticOutputParser(pydantic_object=EmailSummary)

### 프롬프트 작성 및 출력포맷 변수 채워넣기 

In [20]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
당신은 유능한 어시스턴트 입니다. 다음 질문에 대해 답변해주세요.

# 사용자 질문
{user_question}

# 이메일
{email}

# 출력 포맷
{format}
"""
)

# format 변수에 PydanticOutputParser 담기 
prompt = prompt.partial(format=parser.get_format_instructions())
print(prompt)

input_variables=['email', 'user_question'] input_types={} partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender": {"description": "발신자", "title": "Sender", "type": "string"}, "sender_email": {"description": "발신자 이메일 주소", "title": "Sender Email", "type": "string"}, "recipient": {"description": "수신자", "title": "Recipient", "type": "string"}, "recipient_email": {"description": "수신자 이메일 주소", "title": "Recipient Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메

### llm 객체 생성 
- with_structured_output()메소드로 모델에서 출력값 구조화 하기 
- LangSmith에서 chain에 parser를 추가해준 것과 어떤 차이가 있는지 확인 해보기

In [21]:
llm = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    temperature=0
    ).with_structured_output(EmailSummary)

### 체인 생성 후 실행

ChatOpenAI > Input에 prompt 변수에 넘겨준 출력 포맷이 함께 들어가 있음

In [22]:
chain = prompt | llm 

response = chain.invoke(
    {
        'user_question': '이메일에서 주요 내용을 추출해 주세요.',
        'email': email
    }
)
response

EmailSummary(sender='박지현', sender_email='Jihyun.Park@abc.com', recipient='김민재', recipient_email='Minjae@def.com', subject='[미팅 제안] 기술 협력을 통한 상호 발전 방안 논의', summary='박지현이 김민재에게 AI 알고리즘과 데이터 분석 기술의 시너지 효과에 대해 논의하기 위해 10월 25일에 미팅을 제안하는 내용의 이메일.', date='10월 25일')

In [23]:
print(response.sender)

박지현


### 체인 생성 없이 실행

ChatOpenAI > Input에 prompt 변수에 넘겨준 출력 포맷이 들어가 있지 않음

In [24]:
llm = ChatOpenAI(
    model_name='gpt-4o-mini', 
    temperature=0
    ).with_structured_output(EmailSummary)

response = llm.invoke(email)
print(response)

sender='박지현' sender_email='Jihyun.Park@abc.com' recipient='김민재' recipient_email='Minjae@def.com' subject='[미팅 제안] 기술 협력을 통한 상호 발전 방안 논의' summary='박지현이 김민재에게 AI 알고리즘과 데이터 분석 기술의 시너지 효과에 대해 논의하기 위한 미팅을 10월 25일에 제안함.' date='2023-10-25'


In [25]:
response.sender

'박지현'

# 3. CommaSeperatedListOutputParser
- 쉼표로 구분된 항목 리스트를 반환
- 인덱싱을 통해 데이터를 다루기 쉬워짐

### 출력 파서 객체 생성

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# 출력 파서 객체 생성
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기 
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


### 프롬프트 템플릿 작성

In [27]:
prompt = PromptTemplate(
    template="{subject}에대해 5개를 한국어로 나열하세요.\n{format_instructions}",
    input_variables=['subject'],
    partial_variables={'format_instructions': format_instructions},
)

print(prompt)

input_variables=['subject'] input_types={} partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'} template='{subject}에대해 5개를 한국어로 나열하세요.\n{format_instructions}'


### llm 객체 생성

In [ ]:
llm = ChatOpenAI(model_name = 'gpt-4o-mini')

### chain생성 및 실행

In [28]:
chain = prompt | llm | output_parser

#### invoke()로 출력

In [29]:
answer = chain.invoke({'subject': '한국 관광명소'})
print(type(answer))
print(answer)

<class 'list'>
['경복궁', '남산타워', '부산 해운대해수욕장', '제주도 성산일출봉', '경주 불국사temples']


#### stream()으로 출력 

In [30]:
# 원칙상으로 잘 안 쓰이긴 함 (리스트가 5개로 나오기 때문) 
for a in chain.stream({'subject': '한국 관광명소'}):
    print(a)

['경복궁']
['남산타워']
['부산 해운대해수욕장']
['제주도 용두암']
['경주 불국사temples']


In [31]:
a_list = []
for a in chain.stream({'subject': '한국 관광명소'}):
    a_list.extend(a)
print(a_list)

['경복궁', '남산타워', '부산 해운대해수욕장', '제주도 용두암', '경주 불국사emple']


# 4. JasonOutputParser()
- 사용자가 원하는 JSON 스키마를 지정할 수 있게 해줌 
- LLM에게 사용자 정의에 맞는 JSON 스키마 출력을 요구할 경우 모델의 용량/크기가 충분히 커야 함 
- JSON 서버와 클라이언트 간의 통신을 위해서 쓰이는 포맷임 
- JSON은 읽기 쉽고 기계가 파싱하고 생성하기 쉬운 텍스트를 기반으로 함 

### llm 객체 생성 

In [36]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

### 출력 포맷 정의

In [37]:
class Dictionary(BaseModel):
    word: str = Field(description='난이도가 높은 영어 단어')
    meaning: str = Field(description='영어 단어(word)의 한국어 의미')
    synonym: str = Field(description='영어 단어(word)의 유사어')
    example: str = Field(description='영어 단어(word)를 활용한 영어문장 예시')
    part: str = Field(description='영어 단어(word)의 품사')

### 프롬프트 작성

#### Chat 형식으로 프롬프트 생성

In [38]:
# 사용자 입력
lyric = """"Hang your head low in the glow of the vending machine
I'm not dying (Oh yeah, you're right, I want it)
We say that we'll just screw it up in these trying times
We're not trying (Oh yeah, you're right, I want it)
So cut the headlights, summer's a knife
I'm always waiting for you just to cut to the bone
Devils roll the dice, angels roll their eyes
And if I bleed, you'll be the last to know
Oh, it's new, the shape of your body
"""

template = """
다음은 노래가사야.
노래가사에서 명사, 동사, 형용사를 뽑아서 사전을 만들거야. 
총 10개의 단어를 뽑아서 사전을 만들어줘.

아래에 명시된 각 명사, 동사, 형용사에서의 제외어는 제외하고 단어를 뽑아줘. 
--제외어--
명사: I, my, me, mine, you, your, yours, it, its
동사: is, are, am

아래에 제시된 포맷으로 출력해줘.
--출력 포맷--
{format_instructions}

--가사--
{lyric}
"""

# 출력 파서 설정
parser = JsonOutputParser(pydantic_object=Dictionary)
print(parser.get_format_instructions())

# 챗프롬프트 객체 생성 
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 영어사전 제작자 입니다.'),
        ('user', template)
    ]
)

# partial로 포맷 지시사항 채워주기 
prompt = prompt.partial(format_instructions = parser.get_format_instructions())

# 체인 생성
chain = prompt | llm | parser
chat_answer = chain.invoke({'lyric': lyric})
chat_answer

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"word": {"description": "\ub09c\uc774\ub3c4\uac00 \ub192\uc740 \uc601\uc5b4 \ub2e8\uc5b4", "title": "Word", "type": "string"}, "meaning": {"description": "\uc601\uc5b4 \ub2e8\uc5b4(word)\uc758 \ud55c\uad6d\uc5b4 \uc758\ubbf8", "title": "Meaning", "type": "string"}, "synonym": {"description": "\uc601\uc5b4 \ub2e8\uc5b4(word)\uc758 \uc720\uc0ac\uc5b4", "title": "Synonym", "type": "string"}, "example": {"description": "\uc601\uc5b4 \ub2e8\uc5b4(word)\ub97c \ud65c\uc6a9\ud55c \uc601\uc5b4\ubb38\uc7a5 \uc608\uc2dc", "title": "Exampl

[{'word': 'head',
  'meaning': '머리 또는 정수리',
  'synonym': 'crown',
  'example': 'He bowed his head in shame.',
  'part': 'noun'},
 {'word': 'glow',
  'meaning': '빛나다, 빛',
  'synonym': 'shine',
  'example': 'The glow of the sunset was beautiful.',
  'part': 'noun'},
 {'word': 'machine',
  'meaning': '기계, 장치',
  'synonym': 'device',
  'example': 'The vending machine dispenses snacks.',
  'part': 'noun'},
 {'word': 'time',
  'meaning': '시간',
  'synonym': 'period',
  'example': "Time flies when you're having fun.",
  'part': 'noun'},
 {'word': 'knife',
  'meaning': '칼',
  'synonym': 'blade',
  'example': 'He used a knife to cut the bread.',
  'part': 'noun'},
 {'word': 'waiting',
  'meaning': '기다림',
  'synonym': 'awaiting',
  'example': 'I am waiting for the bus.',
  'part': 'verb'},
 {'word': 'cut',
  'meaning': '자르다',
  'synonym': 'slice',
  'example': 'Please cut the paper carefully.',
  'part': 'verb'},
 {'word': 'roll',
  'meaning': '구르다, 굴리다',
  'synonym': 'spin',
  'example': 'The ba

In [39]:
print(type(chat_answer))

<class 'list'>


#### PromptTemplate으로 프롬프트 생성

In [40]:
# 사용자 입력
lyric = """"Hang your head low in the glow of the vending machine
I'm not dying (Oh yeah, you're right, I want it)
We say that we'll just screw it up in these trying times
We're not trying (Oh yeah, you're right, I want it)
So cut the headlights, summer's a knife
I'm always waiting for you just to cut to the bone
Devils roll the dice, angels roll their eyes
And if I bleed, you'll be the last to know
Oh, it's new, the shape of your body
"""

template = """
다음은 노래가사야.
노래가사에서 명사, 동사, 형용사를 뽑아서 사전을 만들거야. 
총 10개의 단어를 뽑아서 사전을 만들어줘.

아래에 명시된 각 명사, 동사, 형용사에서의 제외어는 제외하고 단어를 뽑아줘. 
--제외어--
명사: I, my, me, mine, you, your, yours, it, its
동사: is, are, am

아래에 제시된 포맷으로 출력해줘.
--출력 포맷--
{format_instructions}

--가사--
{lyric}
"""

# 출력 파서 설정
parser = JsonOutputParser(pydantic_object=Dictionary)
# print(parser.get_format_instructions())

# 챗프롬프트 객체 생성 
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(template)

# partial로 포맷 지시사항 채워주기 
prompt = prompt.partial(format_instructions = parser.get_format_instructions())

# 체인 생성
chain = prompt | llm | parser
answer = chain.invoke({'lyric': lyric})
answer

[{'word': 'head',
  'meaning': '머리',
  'synonym': '두부',
  'example': 'He nodded his head in agreement.',
  'part': '명사'},
 {'word': 'glow',
  'meaning': '빛나다',
  'synonym': '빛',
  'example': 'The glow of the sunset was beautiful.',
  'part': '명사'},
 {'word': 'vending machine',
  'meaning': '자판기',
  'synonym': '판매기',
  'example': 'I bought a drink from the vending machine.',
  'part': '명사'},
 {'word': 'dying',
  'meaning': '죽어가는',
  'synonym': '소멸하는',
  'example': 'The dying light of the day was fading.',
  'part': '형용사'},
 {'word': 'trying',
  'meaning': '힘든',
  'synonym': '어려운',
  'example': 'These are trying times for everyone.',
  'part': '형용사'},
 {'word': 'cut',
  'meaning': '자르다',
  'synonym': '베다',
  'example': 'Please cut the paper carefully.',
  'part': '동사'},
 {'word': 'waiting',
  'meaning': '기다리다',
  'synonym': '대기하다',
  'example': 'I am waiting for the bus.',
  'part': '동사'},
 {'word': 'roll',
  'meaning': '구르다',
  'synonym': '굴리다',
  'example': 'The ball will roll down the

In [41]:
print(type(answer))
print(type(answer[0]))

<class 'list'>
<class 'dict'>


# 5. PandasDataFrameOutputParser

- langchain 0.1 버전에서 쓰임

In [4]:
import pprint 
from typing import Any, Dict 

import pandas as pd 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PandasDataFrameOutputParser

1. OpenAI 객체 생성

In [6]:
# DataFrame 출력포맷은 상위버전 모델이 잘 동작이 안 된다고 함 
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 

2. pandas로 csv 데이터 로드 

In [7]:
df = pd.read_csv('./data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


3. parser로 csv 데이터로드

In [8]:
# 파서 설정
parser = PandasDataFrameOutputParser(dataframe=df)
# 지시사항 출력 
print(parser.get_format_instructions()) # 데이터프레임을 쓰기 위한 지침이 정의돼있음 

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

4. 컬럼에 대해 값 조회

In [9]:
# 칼럼 작업 예시 
user_query = 'Name칼럼을 조회해주세요.'
template = """
사용자의 질문에 대답하세요.

{format_instructions}

--사용자 질문--
{query}
"""

# 프롬프트 작성
prompt = PromptTemplate(
    template = template,
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

5. chain 생성 및 실행
    - 작동 원리 : llm이 사용자 입력을 보고 dataframe에 실행할 쿼리문을 생성해서 실행 후 그 결과를 반환해줌 
    - Dataframe을 입력값으로 주고 그걸 보고 판단을 해서 결과를 주는 게 아니라 llm은 사용자의 질문만 쿼리로 바꿔서 parser객체에 준 df에서 실행 시킨 후 그 결과르 반환하게 됨

In [10]:
chain = prompt | llm | parser 

parser_output = chain.invoke({'query': user_query})
parser_output

{'Name': 0                                Braund, Mr. Owen Harris
 1      Cumings, Mrs. John Bradley (Florence Briggs Th...
 2                                 Heikkinen, Miss. Laina
 3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
 4                               Allen, Mr. William Henry
                              ...                        
 886                                Montvila, Rev. Juozas
 887                         Graham, Miss. Margaret Edith
 888             Johnston, Miss. Catherine Helen "Carrie"
 889                                Behr, Mr. Karl Howell
 890                                  Dooley, Mr. Patrick
 Name: Name, Length: 891, dtype: object}

In [11]:
print(type(parser_output))
print(type(parser_output['Name']))

<class 'dict'>
<class 'pandas.core.series.Series'>


In [12]:
print(parser_output['Name'][0])

Braund, Mr. Owen Harris
